In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'C:\Users\Nirmal\Desktop\india-news-headlines.csv')

In [3]:
df

,publish_date,headline_category,headline_text
0,20010102,unknown,Status quo will not be disturbed at Ayodhya; s...
1,20010102,unknown,Fissures in Hurriyat over Pak visit
2,20010102,unknown,America's unwanted heading for India?
3,20010102,unknown,For bigwigs; it is destination Goa
4,20010102,unknown,Extra buses to clear tourist traffic
...,...,...,...
3424062,20201231,city.jodhpur,Covid-19: Despite dip in cases; Rajasthan amon...
3424063,20201231,city.udaipur,Covid-19: Despite dip in cases; Rajasthan amon...
3424064,20201231,city.ajmer,Covid-19: Despite dip in cases; Rajasthan amon...
3424065,20201231,removed,Govt extends deadline for use of FASTag till F...


In [7]:
df['headline_text'][2]

"America's unwanted heading for India?"

# Data Preparation

In [9]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

# Stemming

In [10]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def stemmer(text):
    return [porter.stem(word) for word in text.split()]

In [11]:
stemmer(df['headline_text'][0])

['statu',
 'quo',
 'will',
 'not',
 'be',
 'disturb',
 'at',
 'ayodhya;',
 'say',
 'vajpaye']

# Vectorizing of document


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents = None, lowercase = False, tokenizer = stemmer, use_idf = True, norm ='l2',smooth_idf = True )

In [17]:
Y = df.headline_text.values
X = tfidf.fit_transform(df.headline_text)

# Classification using Logistic Regression

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0, test_size=0.5, shuffle=False)

In [19]:
import pickle
from sklearn.linear_model import LogisticRegressionCV

logit = LogisticRegressionCV(cv=5, scoring='accuracy', max_iter=100)
logit.fit(X_train,y_train)

C:\Users\Nirmal\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


KeyboardInterrupt: 

In [ ]:
filename = 'lr.sav'
pickle.dump(logit, open(filename, 'wb'))

In [ ]:
logit = pickle.load(open(filename, 'rb'))
# get_accuracy_metrics(y_test, logit.predict(X_test))

In [ ]:
logit.score(X_test,y_test)

In [ ]:
y_pred = logit.predict(X_test)

In [ ]:
def get_accuracy_metrics(y_test, y_hat):

  #Generating accuracy score
  print("\nAccuracy attained : {}\n".format(accuracy_score(y_hat,y_test)))

  #Getting the classification matrix
  print("The classification report is :\n\n{}".format(classification_report(y_test, y_hat)))

  #Confusion matrix
  # print("Confusion matrix generated"confusion_matrix(y_test, y_hat))
  print("\n")
  sns.heatmap(confusion_matrix(y_test, y_hat),annot=True,fmt='g', square=True)

def plot_accuracies(y_hat,model=''):
  x,accs=[],[]
  max_x,max_acc = 0,0

  #iterating through various values of threshold possible ie 0-100
  for i in range(0,105,5):
    
    x.append(i)
    z=[]
    
    #checking if the probability is greater than threshold for each y_hat predicted
    for row in y_hat*100:
      if max(row)>i:
        z.append(np.argmax(row))
      else :
        z.append(np.argmin(row))
    
    #Generating a list for accuracy scores
    accs.append(accuracy_score(z,y_test))
    
    if accuracy_score(z,y_test) >= max_acc : 
      max_acc = accuracy_score(z,y_test)
      max_x = i
    else : 
      continue

  #Plotting function
  plt.figure(figsize=(10,5))
  plt.plot(x,accs)
  plt.axvline(x=max_x,label='Maximum accuracy at x = {}, value is {}'.format(max_x, max_acc), color='red')
  plt.title("Accuracies of various thresholds of {}".format(model))
  plt.xlabel("Value of threshold")
  plt.ylabel("Accuracy")
  plt.legend()

  return accs,x

In [ ]:
import numpy    as np
import pandas   as pd
import seaborn  as sns

import pickle
import matplotlib.pyplot as plt

from sklearn.preprocessing    import StandardScaler
from sklearn.preprocessing    import RobustScaler
from sklearn.preprocessing    import MinMaxScaler
from sklearn.model_selection  import train_test_split
from sklearn.metrics          import accuracy_score
from sklearn.metrics          import classification_report
from sklearn.metrics          import confusion_matrix

from sklearn.decomposition  import PCA
get_accuracy_metrics(y_test, y_pred)